# Model Fitting
Purpose of this script is to use the data formatted in 01 to fit a grouped lasso model to the data, and make predictions on the test data.

In [1]:
options(stringsAsFactors = FALSE)
library(dplyr, quietly = TRUE, warn.conflicts = FALSE)
library(grplasso, quietly = TRUE)

Warning message:
"package 'dplyr' was built under R version 3.3.2"Warning message:
"package 'grplasso' was built under R version 3.3.2"

In [2]:
# Read in data from 01
load('../intermed-rdatas/m1_objects.RData')
load('../intermed-rdatas/m2_objects.RData')
load('../intermed-rdatas/test_objects.RData')

train <- read.csv('../data/train.csv')
test <- read.csv('../data/test.csv')

# What have we just imported?
ls()

[1] "cat_mm_test"   "grp_ind_1"     "grp_ind_2"     "num_vars_test"
[5] "test"          "train"         "x1"            "x2"           
[9] "y1"

### Partition Test Data
Not all test data will be used to fit model 1.  Many test observations have levels of variables X0, X2, and X5 that are not in the training data.  We will also need to fill in columns with zeros that are missing from the test data but included in the trained model.

In [3]:
for_m1 <- (test$X0 %in% unique(train$X0)) &
          (test$X2 %in% unique(train$X2)) &
          (test$X5 %in% unique(train$X5))

table(for_m1)

for_m1
FALSE  TRUE 
   24  4185 

In [4]:
# Keep rows for use in m1
x1_test <- cbind(1, cat_mm_test, num_vars_test)[for_m1,]
# Keep cols that are in x1
x1_test <- x1_test[colnames(x1_test) %in% colnames(x1)]
# Fill remainders with 0
for (cn in setdiff(colnames(x1), colnames(x1_test))) x1_test[cn] <- 0
# Reorder to match x1
x1_test <- x1_test[colnames(x1)]
    
# Repeat for model 2
x2_test <- cbind(1, cat_mm_test, num_vars_test)[!for_m1,]
x2_test <- x2_test[colnames(x2_test) %in% colnames(x2)]
for (cn in setdiff(colnames(x2), colnames(x2_test))) x2_test[cn] <- 0
x2_test <- x2_test[colnames(x2)]

### Fit Models
Finally!

In [5]:
lambda <- 10
system.time({
    m1 <- grplasso(x1, y1, index = grp_ind_1, lambda = lambda, model = LinReg())
    m2 <- grplasso(x2, y1, index = grp_ind_2, lambda = lambda, model = LinReg())
})

Warning message in grplasso.default(x1, y1, index = grp_ind_1, lambda = lambda, :
"Penalization not adjusted to non-penalized predictors."Warning message in grplasso.default(x1, y1, index = grp_ind_1, lambda = lambda, :
"Maximal number of iterations reached for lambda[1]"

Lambda: 10  nr.var: 307 


Warning message in grplasso.default(x2, y1, index = grp_ind_2, lambda = lambda, :
"Penalization not adjusted to non-penalized predictors."Warning message in grplasso.default(x2, y1, index = grp_ind_2, lambda = lambda, :
"Maximal number of iterations reached for lambda[1]"

Lambda: 10  nr.var: 231 


   user  system elapsed 
 353.84    0.78  364.10 

In [6]:
yhat_test1 <- exp(predict(m1, x1_test))
yhat_test2 <- exp(predict(m2, x2_test))

In [7]:
final_preds <- rep(0, nrow(test))
final_preds[for_m1] <- yhat_test1
final_preds[!for_m1] <- yhat_test2

In [8]:
df_out <- data.frame(ID = test$ID, y = final_preds)

In [9]:
write.csv(df_out, '../output/lambda10.csv', row.names = FALSE)

## Conclusion
For various values of $\lambda$, my testing score (measured in $R^2$) tends to float between 52% and 54%, with my high score being almost 55%.  The high score on the leaderboard is about 57.5%.  In a real life scenario, these would be comparable scores.  On Kaggle, however, this does not even land me in the top 67% of user submissions!  

Again, in practice, this model took me a combined 10 hours to write (the fitting process takes about 5 min).  Judging from user-submitted kernels of participants using neural net-style models, theirs took a lot longer to write, and likely hours to fit.  My models were run locally, as I did not opt to use AWS or any other similar platform.  Given my shoestring-budget model, I am very proud of these results.

I'm very curious to see if any other users scored above 55% using a model that was neither a neural net nor a tree method.  I suspect not 8).

Someone with more time on his or her hands could likely improve my model by:
* Actually using some sort of cross-validation to select the optimal $\lambda$.
* Using different $\lambda$s for `m1` and `m2`.
* The residual plot is actually terrible.  I believe there is some sort of clustering that can be incorporated to renormalize the residuals.